In [1]:
import copy
import datetime
import numpy
import pandas
import plotly

plotly.offline.init_notebook_mode(connected=True)

In [2]:
DOWN_PAYMENT = 400000
PRINCIPAL = 850000
PAYMENTS_PER_YEAR = 12
START_DATE = datetime.date(2018, 12, 1)

In [3]:
def calc(interest_rate, loan_years, monthly_payment, chunk_payments=0):
    principal = PRINCIPAL - chunk_payments
    
    date_range = pandas.date_range(START_DATE, periods=loan_years * PAYMENTS_PER_YEAR, freq='MS')
    date_range.name = "Payment_Date"
    
    df = pandas.DataFrame(index=date_range,columns=['Payment_Due', 'Payment_Principal', 'Payment_Interest', 'Payment_Extra', 'Cumulative_Interest_Paid', 'Cumulative_Principal_Paid', 'Principal_Balance'], dtype='float')
    df.reset_index(inplace=True)
    df.index += 1
    df.index.name = "Period"
    
    df["Payment_Due"] = numpy.pmt(interest_rate / PAYMENTS_PER_YEAR, loan_years * PAYMENTS_PER_YEAR, principal)
    df["Payment_Principal"] = numpy.ppmt(interest_rate / PAYMENTS_PER_YEAR, df.index, loan_years * PAYMENTS_PER_YEAR, principal)
    df["Payment_Interest"] = numpy.ipmt(interest_rate / PAYMENTS_PER_YEAR, df.index, loan_years * PAYMENTS_PER_YEAR, principal)
    if abs(monthly_payment) > max(abs(df["Payment_Due"])):
        df["Payment_Extra"] = -(monthly_payment - df["Payment_Due"].abs())
    else:
        df["Payment_Extra"] = 0
        monthly_payment = round(max(abs(df["Payment_Due"])))
    df["Cumulative_Interest_Paid"] = df["Payment_Interest"].cumsum().abs()
    df["Cumulative_Principal_Paid"] = (df["Payment_Principal"] + df["Payment_Extra"]).cumsum().abs()
    df["Principal_Balance"] = principal - df["Cumulative_Principal_Paid"]
    df = df.round(2)
    
    sold_off_idxs = (df["Cumulative_Principal_Paid"] > principal)
    
    df = df.drop(df.index[sold_off_idxs == True])
    
    # calculate facts
    facts = {}
    facts['principal'] = principal
    facts['chunk_payments'] = chunk_payments
    facts['interest_rate'] = interest_rate
    facts['monthly_payment'] = monthly_payment
    facts['sold_off'] = {}
    facts['sold_off']['total_months'] = sold_off_idxs[::-1].idxmin()
    facts['sold_off']['years'] = int(facts['sold_off']['total_months'] / 12)
    facts['sold_off']['years_months'] = facts['sold_off']['total_months'] % 12
    facts['total_interest_paid'] = round(max(df["Cumulative_Interest_Paid"]))
    facts['total_paid'] = principal + chunk_payments + facts['total_interest_paid']
    
    return copy.deepcopy(facts), copy.deepcopy(df)

In [4]:
def graph(facts, df):
    data = []
    column_names = list(df.columns.values)
    column_names.sort()
    for column_name in column_names:
        if column_name == 'Payment_Date':
            continue
        data.append( 
            plotly.graph_objs.Scatter(
                x=df.Payment_Date, 
                y=df[column_name], 
                name=column_name
            )
        )

    layout = plotly.graph_objs.Layout(
        title=
                'interest rate: {interest}%<br>' \
                'monthly mortage payment: ${monthly} (~{monthly_total} total)<br>' \
                'paid off after: {years} years {months} months<br>' \
                'total paid: {total_paid} ({down_payment} + {principal} + {chunk_payments} + {total_interest})<br>' \
                'interest paid: {total_interest}'.format(
            interest=facts['interest_rate']*100.0,
            monthly="{:,}".format(facts['monthly_payment']),
            monthly_total="{:,}".format(facts['monthly_payment']+2000),
            years=facts['sold_off']['years'],
            months=facts['sold_off']['years_months'],
            total_paid="{:,}".format(facts['total_paid']+400000),
            down_payment="{:,}".format(DOWN_PAYMENT),
            principal="{:,}".format(facts['principal']),
            chunk_payments="{:,}".format(facts['chunk_payments']),
            total_interest="{:,}".format(facts['total_interest_paid']),
        ),
        width=800,
        height=500,
    )

    fig = plotly.graph_objs.Figure(data=data, layout=layout)
    
    return fig

In [5]:
def calc_and_graph(interest_rate, loan_years, monthly_payment, chunk_payments=0):
    facts, df = calc(interest_rate, loan_years, monthly_payment, chunk_payments)
    fig = graph(facts, df)
    plotly.offline.iplot(fig)

In [6]:
facts, df = calc(0.0425, 30, 8000)

In [7]:
calc_and_graph(interest_rate=0.0425, loan_years=30, monthly_payment=0)

In [8]:
calc_and_graph(interest_rate=0.0425, loan_years=30, monthly_payment=4000, chunk_payments=100000)

In [9]:
calc_and_graph(interest_rate=0.0425, loan_years=30, monthly_payment=6000)

In [10]:
calc_and_graph(interest_rate=0.0425, loan_years=30, monthly_payment=6000, chunk_payments=200000)

In [11]:
calc_and_graph(interest_rate=0.04, loan_years=15, monthly_payment=0)

In [12]:
calc_and_graph(interest_rate=0.04, loan_years=15, monthly_payment=0, chunk_payments=200000)